# Chat Completion API

In [1]:
!pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google - colab - secret 탭에 'OPENAI_API_KEY' 등록

In [7]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [9]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [13]:

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 정확하고 전문적인 챗봇이야."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕 나는 차은우야"
        }
      ]
    },
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [14]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스 AFP 속보시스템에서 도입되어 사용

In [21]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ### => 아래처럼 예시 2개면 2-shot-prompting
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 테이의 FM 개꿀 라디오 방송에 주목해주세요.
- 교정 제목: 테이의 FM 라디오 방송에 주목하세요.


In [29]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
                },
                {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
                }
            ],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말에 미친 개같은 폭우 예상.. 모두들 무사하시길 바래용'
output = correct_news_title(title_before, temperature=1, top_p=1)
print(output)

- 원래 제목: 주말에 미친 개같은 폭우 예상.. 모두들 무사하시길 바래용
- 교정 제목: 주말, 강한 폭우 예보… 모두 안전에 유의하시기 바랍니다.


##  영문텍스트에 anki 단어장 만들기

In [31]:
# 함수 리팩토링

def anki_create(input_text, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    당신은 영어 단어 학습을 위한 Anki 단어장 생성 도우미입니다.

사용자가 제공한 영어 문장이나 단락에서 주요 어휘를 선별하고, 각 단어를 암기하기 좋은 카드 형태로 정리하세요.

### 카드 구성 방식 ###
각 단어에 대해 다음 정보를 포함하세요:

- 단어: [영단어]
- 의미: [한국어 의미]
- 품사: [명사/동사/형용사 등]
- 예문: [원문에서의 문장 또는 간단한 예시 문장]
- 발음기호(IPA): [가능하면]

### 출력 형식 ###
각 단어마다 아래와 같은 형식으로 출력하세요.

---
단어: benevolent
의미: 자비로운, 인정 많은
품사: 형용사
예문: She was a benevolent leader who helped everyone.
예문해석: 그녀는 모두를 도왔던 자비로운 리더였다.
발음기호: /bəˈnevələnt/
---

※ 너무 쉬운 단어(I, the, and 등)는 생략하세요.
※ 총 단어 수는 5~10개 이내로 제한하세요 (너무 많지 않게).
※ 예문에 대한 해석도 달아주세요.


    """

    user_message = f"""
    다음 영어 문장에서 주요 단어를 추출하고, Anki 단어장 형식으로 만들어 주세요.

텍스트:
{input_text}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
                },
                {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
                }
            ],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

input_text = """
"The government implemented new policies to mitigate the effects of climate change."

"Her benevolent nature made her beloved by everyone in the village."

"Despite the challenges, he remained resilient and optimistic."

"The manager emphasized the importance of clear communication in the workplace."

"This book offers a profound insight into the complexities of human behavior."

"""
output = anki_create(input_text, temperature=1, top_p=1)
print(output)

---
단어: implement  
의미: 시행하다, 실행하다  
품사: 동사  
예문: The government implemented new policies to mitigate the effects of climate change.  
예문해석: 정부는 기후 변화의 영향을 완화하기 위해 새로운 정책을 시행했다.  
발음기호: /ˈɪmplɪˌmɛnt/  
---

---
단어: mitigate  
의미: 완화하다, 경감하다  
품사: 동사  
예문: The government implemented new policies to mitigate the effects of climate change.  
예문해석: 정부는 기후 변화의 영향을 완화하기 위해 새로운 정책을 시행했다.  
발음기호: /ˈmɪtɪˌɡeɪt/  
---

---
단어: benevolent  
의미: 자비로운, 인정 많은  
품사: 형용사  
예문: Her benevolent nature made her beloved by everyone in the village.  
예문해석: 그녀의 자비로운 성격은 마을의 모든 사람이 그녀를 사랑하게 만들었다.  
발음기호: /bəˈnɛvələnt/  
---

---
단어: resilient  
의미: 회복력이 있는, 탄력 있는  
품사: 형용사  
예문: Despite the challenges, he remained resilient and optimistic.  
예문해석: 도전에도 불구하고 그는 회복력 있고 낙관적이었다.  
발음기호: /rɪˈzɪljənt/  
---

---
단어: emphasize  
의미: 강조하다  
품사: 동사  
예문: The manager emphasized the importance of clear communication in the workplace.  
예문해석: 관리자는 직장에서 명확한 커뮤니케이션의 중요성을 강조했다.  
발음기호: /ˈɛmfəˌsaɪz/  
---

---
단어: profound  
